In [1]:
from math import *
import random
import time
from tkinter import *

######################################################################
# This section contains functions for loading CSV (comma separated values)
# files and convert them to a dataset of instances.
# Each instance is a tuple of attributes. The entire dataset is a list
# of tuples.
######################################################################

# Loads a CSV files into a list of tuples.
# Ignores the first row of the file (header).
# Numeric attributes are converted to floats, nominal attributes
# are represented with strings.
# Parameters:
#   fileName: name of the CSV file to be read
# Returns: a list of tuples
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    dataset = []
    for line in lines:
        instance = lineToTuple(f'x{len(dataset)},'+line)
        dataset.append(instance)
    return dataset

# Converts a comma separated string into a tuple
# Parameters
#   line: a string
# Returns: a tuple
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

# Destructively converts all the string elements representing numbers
# to floating point numbers.
# Parameters:
#   myList: a list of strings
# Returns None
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

# Checks if a given string can be safely converted into a positive float.
# Parameters:
#   s: the string to be checked
# Returns: True if the string represents a positive float, False otherwise
def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True


######################################################################
# This section contains functions for clustering a dataset
# using the k-means algorithm.
######################################################################

def distance(instance1, instance2, metric):
    if instance1 == None or instance2 == None:
        return float("inf")
    dist = 0
    a, b = 0, 0
    for i in range(1, len(instance1)):
        if(isinstance(instance1[i], str) or isinstance(instance2[i], str)):
            continue
        if(metric == 'euclidean'):
            dist += (instance1[i] - instance2[i])**2
        elif(metric == 'manhattan'):
            dist += abs(instance1[i] - instance2[i])
        elif(metric == 'jacard'):
            a += min(instance1[i], instance2[i])
            b += max(instance1[i], instance2[i])
        elif(metric == 'cosine'):
            dist += instance1[i]*instance2[i]
            a += instance1[i]*instance1[i]
            b += instance2[i]*instance2[i]
    if(metric == 'jacard'):
        dist = 1-a/b
    elif(metric == 'cosine'):
        dist = 1-dist/sqrt(a*b)
    return dist

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    abc = {}
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            if(isinstance(instance[i], str)):
                if(not instance[i] in abc):
                    abc[instance[i]] = 0
                abc[instance[i]] += 1
                means[i] = MaxKey = max(abc, key=abc.get)
                continue
            means[i] += instance[i]
    for i in range(1, numAttributes):
        if(isinstance(instance[i], str)):
            continue
        means[i] /= float(numInstances)
    return tuple(means)

def assign(instance, centroids, metric):
    minDistance = distance(instance, centroids[0], metric)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i], metric)
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def assignAll(instances, centroids, metric):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids, metric)
        clusters[clusterIndex].append(instance)
    return clusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def kmeans(instances, k, metric, iters=-1, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    iteration = 0
    prevwith = 1e70
    while (centroids != prevCentroids or iters < -2):
        iteration += 1
        clusters = assignAll(instances, centroids, metric)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, metric)
        if(iteration == iters or (iters < -2 and iteration == -iters)):
            break
        if(iters == -2 and withinss > prevwith):
            break
        prevwith = withinss
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["iterations"] = iteration
    return result

def computeWithinss(clusters, centroids, metric):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += distance(centroid, instance, metric)**2
    return result

# Task 1

In [2]:
data = [["x1", 3, 5], ["x2", 3, 4], ["x3", 2, 8],
        ["x4", 2, 3], ["x5", 6, 2], ["x6", 6, 4],
        ["x7", 7, 3], ["x8", 7, 4], ["x9", 8, 5], ["x10", 7, 6]]

## Q1

In [3]:
print(kmeans(data, 2, 'manhattan', 1, [['c1', 4, 6], ['c2', 5, 4]]))
print(kmeans(data, 2, 'manhattan', -1, [['c1', 4, 6], ['c2', 5, 4]]))

{'clusters': [[['x1', 3, 5], ['x3', 2, 8], ['x10', 7, 6]], [['x2', 3, 4], ['x4', 2, 3], ['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5]]], 'centroids': [('centroid0', 4.0, 6.333333333333333), ('centroid1', 5.571428571428571, 3.5714285714285716)], 'withinss': 83.22448979591836, 'iterations': 1}
{'clusters': [[['x1', 3, 5], ['x3', 2, 8], ['x10', 7, 6]], [['x2', 3, 4], ['x4', 2, 3], ['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5]]], 'centroids': [('centroid0', 4.0, 6.333333333333333), ('centroid1', 5.571428571428571, 3.5714285714285716)], 'withinss': 83.22448979591836, 'iterations': 2}


## Q2

In [4]:
print(kmeans(data, 2, 'euclidean', 1, [['c1', 4, 6], ['c2', 5, 4]]))
print(kmeans(data, 2, 'euclidean', -1, [['c1', 4, 6], ['c2', 5, 4]]))

{'clusters': [[['x1', 3, 5], ['x3', 2, 8]], [['x2', 3, 4], ['x4', 2, 3], ['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 2.5, 6.5), ('centroid1', 5.75, 3.875)], 'withinss': 387.509765625, 'iterations': 1}
{'clusters': [[['x1', 3, 5], ['x2', 3, 4], ['x3', 2, 8], ['x4', 2, 3]], [['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 2.5, 5.0), ('centroid1', 6.833333333333333, 4.0)], 'withinss': 150.625, 'iterations': 3}


## Q3

In [5]:
print(kmeans(data, 2, 'manhattan', 1, [['c1', 3, 3], ['c2', 8, 3]]))
print(kmeans(data, 2, 'manhattan', -1, [['c1', 3, 3], ['c2', 8, 3]]))

{'clusters': [[['x1', 3, 5], ['x2', 3, 4], ['x3', 2, 8], ['x4', 2, 3]], [['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 2.5, 5.0), ('centroid1', 6.833333333333333, 4.0)], 'withinss': 40.5, 'iterations': 1}
{'clusters': [[['x1', 3, 5], ['x2', 3, 4], ['x3', 2, 8], ['x4', 2, 3]], [['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 2.5, 5.0), ('centroid1', 6.833333333333333, 4.0)], 'withinss': 40.5, 'iterations': 2}


## Q4

In [6]:
print(kmeans(data, 2, 'manhattan', 1, [['c1', 3, 2], ['c2', 4, 8]]))
print(kmeans(data, 2, 'manhattan', -1, [['c1', 3, 2], ['c2', 4, 8]]))

{'clusters': [[['x1', 3, 5], ['x2', 3, 4], ['x4', 2, 3], ['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4]], [['x3', 2, 8], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 4.857142857142857, 3.5714285714285716), ('centroid1', 5.666666666666667, 6.333333333333333)], 'withinss': 96.25850340136054, 'iterations': 1}
{'clusters': [[['x1', 3, 5], ['x2', 3, 4], ['x4', 2, 3], ['x5', 6, 2], ['x6', 6, 4], ['x7', 7, 3], ['x8', 7, 4]], [['x3', 2, 8], ['x9', 8, 5], ['x10', 7, 6]]], 'centroids': [('centroid0', 4.857142857142857, 3.5714285714285716), ('centroid1', 5.666666666666667, 6.333333333333333)], 'withinss': 96.25850340136054, 'iterations': 2}


# Task 2

In [7]:
iris = loadCSV('iris.data')

## Q1

In [8]:
euclid = kmeans(iris, 3, metric='euclidean')
cosine = kmeans(iris, 3, 'cosine')
jacard = kmeans(iris, 3, metric='jacard')
print(euclid['withinss'], cosine['withinss'], jacard['withinss'])


84.83745513407887 0.0003446871125918868 1.0452360489145653


## Q2

In [9]:
def acc(res):
    ac = 0
    tot = 0
    for i in range(len(res['centroids'])):
        for j in res['clusters'][i]:
            if(j[5] == res['centroids'][i][5]):
                ac += 1
            tot += 1
    return ac/tot
print(acc(euclid), acc(cosine), acc(jacard))

0.8933333333333333 0.9666666666666667 0.88


## Q3

In [10]:

print(euclid['iterations'], cosine['iterations'], jacard['iterations'])

4 4 4


## Q4

In [18]:
def test(metric):
    print(metric, kmeans(iris, 3, metric, -1)['iterations'], kmeans(iris, 3, metric, -2)['iterations'], kmeans(iris, 3, metric, -100)['iterations'])
test('euclidean')
test('cosine')
test('jacard')

euclidean 6 13 100
cosine 5 2 100
jacard 5 6 100


# Task 4

In [12]:
import numpy as np

red = [[4.7, 3.2], [4.9, 3.1], [5.0, 3.0], [4.6, 2.9]]
blue = [[5.9, 3.2], [6.7, 3.1], [6.0, 3.0], [6.2, 2.8]]
redDist = [[hypot(red[i][0]-red[j][0], red[i][1]-red[j][1]) for j in range(i)] for i in range(1, 4)]
blueDist = [[hypot(blue[i][0]-blue[j][0], blue[i][1]-blue[j][1]) for j in range(i)] for i in range(1, 4)]

def apply(x):
    return x([x(map(x, redDist)), x(map(x, blueDist))])
print(redDist)
print(blueDist)

[[0.22360679774997919], [0.36055512754639885, 0.1414213562373093], [0.3162277660168384, 0.36055512754639957, 0.4123105625617664]]
[[0.8062257748298547], [0.22360679774997896, 0.7071067811865478], [0.5000000000000002, 0.5830951894845302, 0.2828427124746193]]


## Q1

In [13]:
apply(max)

0.8062257748298547

## Q2

In [14]:
apply(min)

0.1414213562373093

## Q3

In [15]:
apply(sum)/12

0.4097961661153519